# Исследование надежности заемщиков


## Откроем таблицу и изучим общую информацию о данных

**Импортируем библиотеку pandas. Считаем данные из csv-файла в датафрейм и сохраним в переменную `data`.**

In [1]:
import pandas as pd

try:
    data = pd.read_csv('/datasets/data.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

**Выведем первые 20 строчек датафрейма `data` на экран.**

In [2]:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


**Выведем основную информацию о датафрейме с помощью метода `info()`.**

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Предобработка данных

### Удаление пропусков

**Выведем количество пропущенных значений для каждого столбца**

In [4]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

В двух столбцах есть пропущенные значения. Один из них — `days_employed`. Другой столбец с пропущенными значениями — `total_income` — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполним пропуски в этом столбце медианным значением по каждому типу из столбца `income_type`. Например, у человека с типом занятости `сотрудник` пропуск в столбце `total_income` будет заполнен медианным доходом среди всех записей с тем же типом.

In [5]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Обработка аномальных значений

В данных могут встречаются артефакты (аномалии) — значения, которые не отражают действительность и появились по какой-то ошибке. Таким артефактом будет отрицательное количество дней трудового стажа в столбце `days_employed`. Заменим все отрицательные значения положительными с помощью метода `abs()`

In [6]:
data['days_employed'] = data['days_employed'].abs()

**Для каждого типа занятости выведем медианное значение трудового стажа `days_employed` в днях**

In [7]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

**Выведем перечень уникальных значений столбца `children`**

In [8]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

**В столбце `children` есть два аномальных значения. Удалим строки, в которых встречаются такие аномальные значения из датафрейма `data`**

In [9]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

**Ещё раз выведем перечень уникальных значений столбца `children`, чтобы убедиться, что артефакты удалены**

In [10]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

### Удаление пропусков (продолжение)

**Заполним пропуски в столбце `days_employed` медианными значениями по каждого типа занятости `income_type`**

In [11]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

**Убедимся, что все пропуски заполнены. Проверим себя и ещё раз выведем количество пропущенных значений для каждого столбца**

In [12]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Изменение типов данных

**Заменм вещественный тип данных в столбце `total_income` на целочисленный с помощью метода `astype()`.**

In [13]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

**Обработаем неявные дубликаты в столбце `education`. В этом столбце есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведем их к нижнему регистру. Проверим остальные столбцы.**

In [14]:
data['education'] = data['education'].str.lower()

**Выведем на экран количество строк-дубликатов в данных. Если такие строки присутствуют, удалим их.**

In [15]:
data.duplicated().sum()

71

In [16]:
data = data.drop_duplicates()

### Категоризация данных

**Задание 16. На основании диапазонов, указанных ниже, создадим в датафрейме `data` столбец `total_income_category` с категориями:**

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.


**Например, кредитополучателю с доходом 25000 нужно назначить категорию `'E'`, а клиенту, получающему 235000, — `'B'`. Используем собственную функцию с именем `categorize_income()` и метод `apply()`.**

In [17]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [18]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

**Выведем на экран перечень уникальных целей взятия кредита из столбца `purpose`.**

In [19]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

**Создадим функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:**

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

**Например, если в столбце `purpose` находится подстрока `'на покупку автомобиля'`, то в столбце `purpose_category` должна появиться строка `'операции с автомобилем'`.**

**Используем собственную функцию с именем `categorize_purpose()` и метод `apply()`.**

In [20]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [21]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

## Исследуем данные и ответим на вопросы

### Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [22]:
#функция по разделению заемщиков на категории в зависимости от количества детей
def categorize_children(children):
    try:
        if children == 0:
            return "без детей"
        elif children == 1:
            return "1 ребенок"
        elif children == 2:
            return "2 ребенка"
        elif children >= 3:
            return "3 и более детей"
    except:
        pass

#категорирование данных по количеству детей    
data['children_category'] = data['children'].apply(categorize_children)    

#
#функция для создания и вывода сводной таблицы к заданному df и по нужным категориям и значениям. Ниже описан ее функционал.
#def analytical_table(df, index_in, values_in)

# #функция по преобразованию среднего арифметического значения в проценты (используется в pivot_tabel вместо функции mean)    
#def make_percent_mean(series):
#    return round(series.mean()*100, 1).astype(str) + '%'

# #преобразование любого столбца в проценты
# def make_percent_parts(series_part):
#     return round(series_part * 100, 1).astype(str) + '%'

# #создание сводной таблицы
# data_pivot = data.pivot_table(index=[index_in], values=values_in, aggfunc=['count', 'sum', make_percent_mean], margins=True)

# #создание столбца с долей кредитов по категориям (общее количество)
# data_pivot['part_all_credits'] = make_percent_parts(df[index_in].value_counts() / df[index_in].count())

# #заполнение пропуска в итоговой строке на 100%
# data_pivot['part_all_credits'] = data_pivot['part_all_credits'].fillna('100%')

# #переименование столбцов сводной таблицы: соединение мультииндекса в один и затем присвоение нового имени
#data_pivot.columns = ["".join((a, b)) for a, b in data_pivot.columns]
#data_pivot = data_pivot.rename(columns={'countdebt': 'credits_count', 'sumdebt': 'delay_count', 'makepercent_meandebt': 'percentage_of_delay', 'make_percent_meandebt': 'percentage_of_delay'})

# #вывод сводной таблицы
# data_pivot
#

def analytical_table(df, index_in, values_in):
    def make_percent_mean(series):
        return round(series.mean()*100, 1).astype(str) + '%'
    def make_percent_parts(series_part):
        return round(series_part * 100, 1).astype(str) + '%'
    data_pivot = df.pivot_table(index=[index_in], values=values_in, aggfunc=['count', 'sum', make_percent_mean], margins=True)
    data_pivot['part_all_credits'] = make_percent_parts(df[index_in].value_counts() / df[index_in].count())
    data_pivot['part_all_credits'] = data_pivot['part_all_credits'].fillna('100%')
    data_pivot.columns = ["".join((a, b)) for a, b in data_pivot.columns]
    data_pivot = data_pivot.rename(columns={'countdebt': 'credits_count', 'sumdebt': 'delay_count', 'make_percent_meandebt': \
                                            'percentage_of_delay', 'make_percent_meandebt': 'percentage_of_delay'})
    display(data_pivot)

#вызов функции и вывод сводной таблицы с ее помощью
analytical_table(data, 'children_category', 'debt')



,credits_count,delay_count,percentage_of_delay,part_all_credits
children_category,,,,
1 ребенок,4808,444,9.2%,22.5%
2 ребенка,2052,194,9.5%,9.6%
3 и более детей,380,31,8.2%,1.8%
без детей,14091,1063,7.5%,66.1%
All,21331,1732,8.1%,100%


**Вывод:** 
- больше всего кредитов берут люди без детей (66,1% всех кредитов). 
- меньше всего кредитов берут люди с 3 и более детьми (1,8% всех кредитов).
- меньше всего просрочек допускают люди без детей (по 7,5% кредитов были допущены просрочки, что даже ниже среднего уровня в целом по всем кредитам).
- в целом, процент просрочек несильно отличается в зависимости от количества детей. Разница между минимальной и максимальной долей просрочки составляет 2 п.п.

Т.о. количество детей имеет существенное влияние на количество кредитов, т.к. зависимость в данном случае обратная (меньше детей - больше кредитов). Зависимость возвратности от количества детей незначительная.

### Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [23]:
#функция категорирования на людей состоящих в браке и на тех, кто никогда не был женат/замужем
def family_status_categorize(status):
    try:
        if status == 'Не женат / не замужем' or status == 'гражданский брак':
            return 'Не состояли в браке'
        elif status == 'в разводе' or status == 'вдовец / вдова' or status == 'женат / замужем':
            return 'Состояли в браке'
    
    except:
        pass

#добавление в датафрейм нового столбца с укрупненной категорией семейного положения
data['family_status_category'] = data['family_status'].apply(family_status_categorize)

display(analytical_table(data, 'family_status_category', 'debt'))
display(analytical_table(data, 'family_status', 'debt'))


,credits_count,delay_count,percentage_of_delay,part_all_credits
family_status_category,,,,
Не состояли в браке,6930,658,9.5%,32.5%
Состояли в браке,14401,1074,7.5%,67.5%
All,21331,1732,8.1%,100%


None

,credits_count,delay_count,percentage_of_delay,part_all_credits
family_status,,,,
Не женат / не замужем,2796,273,9.8%,13.1%
в разводе,1189,84,7.1%,5.6%
вдовец / вдова,951,63,6.6%,4.5%
гражданский брак,4134,385,9.3%,19.4%
женат / замужем,12261,927,7.6%,57.5%
All,21331,1732,8.1%,100%


None

**Вывод:** 
- наибольшее количество кредитов приходится на людей, когда-либо состоявших в браке (67,5%);
- люди, которые когда-либо были в браке или состоят в нем имеют наименьший процент просрочки (7,5%);
- люди, никогда не состоявшие в официальном браке допускают просрочку чаще (доля просрочек 9,5%).
- наименьшее количество просрочек допускают вдовцы / вдовы (6,6%), а наимбольшее не женатые / не замужние люди (9,8%).

Т.о., люди, имевшие ранее зарегистрированные отношения допускают просрочек меньше, что, возможно, объясняется их более высоким уровнем ответственности.

### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [24]:
#создадим новую функцию категорирования данных по доходу, т.к. в настоящий момент в категорию "С" попадает слишком большой объем выборки
#отсортируем список по убыванию дохода
data_sorted = data.sort_values(by='total_income', ascending=False)
#найдем медианное значение дохода
data_sorted_total_income_median = data_sorted['total_income'].median()
#выделим людей, имеющих доход ниже медианного (включительно) и найдем там медиану
data_sorted_total_income_lower_middle = data_sorted.loc[data_sorted['total_income'] <= data_sorted_total_income_median]
data_sorted_total_income_lower_middle_median = data_sorted_total_income_lower_middle['total_income'].median()
#выделим людей, имеющих доход выше медианного (включительно) и найдем там медиану
data_sorted_total_income_higher_middle = data_sorted.loc[data_sorted['total_income'] > data_sorted_total_income_median]
data_sorted_total_income_higher_middle_median = data_sorted_total_income_higher_middle['total_income'].median()

#функция категорирования дохода по медианным значениям.
def categorize_income_new(income):
    try:
        if 0 <= income <= 108050.0:
            return 'меньше 108050 рублей'
        elif 108050.0 < income <= 142594.0:
            return '108050.0 - 142594 рублей'
        elif 142594.0 < income <= 196641.5:
            return '142594.0 - 196641.5 рублей'
        elif 196641.5 < income:
            return 'больше 196641.5 рублей'        
    except:
        pass

#создание в основном датафрейме столбца с категорией
data['total_income_category_new'] = data['total_income'].apply(categorize_income_new)

#вывод сводной таблицы с новыми категориями
display(analytical_table(data, 'total_income_category_new', 'debt'))

#вывод сводной таблицы со старыми категориями
display(analytical_table(data, 'total_income_category', 'debt'))

,credits_count,delay_count,percentage_of_delay,part_all_credits
total_income_category_new,,,,
108050.0 - 142594 рублей,5391,475,8.8%,25.3%
142594.0 - 196641.5 рублей,5274,450,8.5%,24.7%
больше 196641.5 рублей,5274,375,7.1%,24.7%
меньше 108050 рублей,5392,432,8.0%,25.3%
All,21331,1732,8.1%,100%


None

,credits_count,delay_count,percentage_of_delay,part_all_credits
total_income_category,,,,
A,25,2,8.0%,0.1%
B,5014,354,7.1%,23.5%
C,15921,1353,8.5%,74.6%
D,349,21,6.0%,1.6%
E,22,2,9.1%,0.1%
All,21331,1732,8.1%,100%


None

**Вывод:** 
- наименьшее количество просрочек допускают люди с высоким доходом - 7,1%
- люди, допускающие просрочки чаще всего, по доходу находятся в середине выборки
- при этом, разница между долей просроченных платежей между группами несущественная (между минимальным и максимальным значением около 2%)

Т.о., зависимость между доходом и уровнем просрочек неочевидна.

### Как разные цели кредита влияют на его возврат в срок?

In [25]:
#создание фукнции для вывода сводной таблицы с подкатегориями
def analytical_table_for(df, index_in_1, index_in_2, values_in):
    def make_percent_mean(series):
        return round(series.mean()*100, 1).astype(str) + '%'
    data_pivot = df.pivot_table(index=[index_in_1, index_in_2], values=values_in, aggfunc=['count', 'sum', make_percent_mean], \
                                margins=True)
    data_pivot.columns = ["".join((a, b)) for a, b in data_pivot.columns]
    data_pivot = data_pivot.rename(columns={'countdebt': 'credits_count', 'sumdebt': 'delay_count', 'make_percent_meandebt': \
                                            'percentage_of_delay', 'make_percent_meandebt': 'percentage_of_delay'})
    display(data_pivot)

#вывод сводной таблицы по целям кредита без подкатегорий (первая таблица)
display(analytical_table(data, 'purpose_category', 'debt'))

#вывод сводной таблицы по целям кредита с подкатегориями (вторая таблица)
display(analytical_table_for(data, 'purpose_category', 'family_status_category', 'debt'))

,credits_count,delay_count,percentage_of_delay,part_all_credits
purpose_category,,,,
операции с автомобилем,4279,400,9.3%,20.1%
операции с недвижимостью,10751,780,7.3%,50.4%
получение образования,3988,369,9.3%,18.7%
проведение свадьбы,2313,183,7.9%,10.8%
All,21331,1732,8.1%,100%


None

credits_count  delay_count  \
purpose_category         family_status_category                               
операции с автомобилем   Не состояли в браке              1065          133   
                         Состояли в браке                 3214          267   
операции с недвижимостью Не состояли в браке              2574          220   
                         Состояли в браке                 8177          560   
получение образования    Не состояли в браке               978          122   
                         Состояли в браке                 3010          247   
проведение свадьбы       Не состояли в браке              2313          183   
All                                                      21331         1732   

                                                percentage_of_delay  
purpose_category         family_status_category                      
операции с автомобилем   Не состояли в браке                  12.5%  
                         Состояли в браке                      8.3%  
операции с недвижимостью Не состояли в браке                   8.5%  
                         Состояли в браке                      6.8%  
получение образования    Не состояли в браке                  12.5%  
                         Состояли в браке                      8.2%  
проведение свадьбы       Не состояли в браке                   7.9%  
All                                                            8.1%

None

**Вывод:** 
- наибольшее количество кредитов получено на цели операций с недвижимостью
- наибольшее количество просрочек отмечается по кредитам на получение образования и на операции с автомобилем
- интересно также категорировать данные дополнительно по семейному положению. Люди в браке допускают просрочек по всем видам кредитов меньше, чем люди, не состоявшие в браке. При этом, люди, которые не состояли в браке, но берут кредит на проведение свадьбы, в последующем допускают просрочек также, как и люди в браке.
- также стоит отметить, что люди, ранее разведенные или вдовы / вдовцы не берут кредитов на проведение свадьбы.

### Приведем возможные причины появления пропусков в исходных данных.

In [26]:
try:
    data_theory = pd.read_csv('/datasets/data.csv')
except:
    data_theory = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

data_theory.isna().sum()


children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Пропуски были выявлены в столбцах days_employed (трудовой стаж) и total_income (общий доход). При чем их количество одинаковое по (2174 штуки). Возможно, имеется некоторая зависимость между указанными столбцами и другими данными, а сами пропуски допущены в одних и тех же строках.

In [27]:
#Отфильтруем данные только по строкам с пропусками в указанных столбцах
data_theory = data_theory.loc[(data_theory['days_employed'].isnull()) & (data_theory['total_income'].isnull())]

#выведем количество строк в датафрейме
len(data_theory)

2174

В отфильтрованном датафрейме, содержащем одновременно пропуски в столбцах days_employed и total_income, столько же строк, сколько и пропусков в основном датафрейме. Логично, что пропуски содержатся в одинаковых строках. 
Т.о., зависимость возникновения пропусков от иных данных таблицы подтверждается.
Попробуем создать сводные таблицы с данными по критерию источника доходов

In [28]:
data_theory['total_income'] = data_theory['total_income'].fillna(0)
data_pivot_theory_income_type = data_theory.pivot_table(index=['income_type'], values='total_income', aggfunc=['count'], \
                                                        margins=True)
data_pivot_theory_income_type

,count
,total_income
income_type,
госслужащий,147
компаньон,508
пенсионер,413
предприниматель,1
сотрудник,1105
All,2174


413 пропусков в данных совершили люди, находящиеся на пенсии. Вероятно, они не имеют иного дохода, кроме пенсии, поэтому решили не указывать данную информацию.
Остальные лица, видимо, просто не пожелали указать данную информацию, либо произошел технический сбой, в результате чего данные не попали в выбоку.

### Объясним, почему заполнить пропуски медианным значением — лучшее решение для количественных переменных.

Заполнение пропущенных данных для количественных переменных медианным значением имеет преимущество по сравнению, например, с заполнением данных по среднему арифметическому значению, т.к. позволяет нивелировать влияние резких выбросов в данных, как то: экстремально высокие, низкие значения.

## Общий вывод.

Было проведено исследование, чтобы выявить влияние разных факторов на факт погашения кредита в срок. Входные данные - статистика о платёжеспособности клиентов. Результаты исследования будут учтены при построении модели кредитного скоринга - специальносй системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.
        
Перед проведением исследования была поставлена задача по выявлению влияния следующих факторов на возврат кредита в срок:     
- **Фактор №1:** Количеством детей;
- **Фактор №2:** Семейное положение;
- **Фактор №3:** Уровень дохода;
- **Фактор №4:** Цель кредита;
        
По итогу исследования выявлены следующие зависимости.
        
**Фактор №1. Влияние фактора подтверждено частично:** Для более точных выводов необходимо больше данных по клиентам с 3, 4 и 5 детьми.
- 7,5% доля просроченных кредитов без детей;
- 9,2% доля просроченных кредитов с 1 ребенком;
- 9,5% доля просроченных кредитов с 2 детьми;
        
Рекомендуется провести исследование на большей выборке данных.
        
**Фактор №2. Влияние фактора подтверждено.**
- Люди, состоявшие в браке ранее (в разводе, вдовец / вдова) или состоящие в браке в настоящий момент, имеют уровень просрочки ниже (7,5%). При чем данный факт подтверждается и в разрезе целей кредитования.
- Люди, никогда не состоявшие в браке (не женат / не замужем, гражданский брак) имеют уровень просрочки в размере 9,5%.
        
Разница в доле просрочки между двумя пунктами 2 п.п. Таким образом, отсутствие зарегистрированных отношений увеличивает количество просроченных кредитов на 26,7%. 
При этом, необходимо учитывать, что данный фактор не является ключевым и единственным, поэтому рекомендуется устанавливать ему невысокий вес при принятии решения об уровне кредитоспособности.
        
**Фактор №3. Влияние фактора подтверждено частично:** Для более точных выводов необходимо больше данных по клиентам с доходом до 50 тыс.руб. (категории E, D), с доходом от 1 млн.руб. (категория А), 
- 7,1% - Наименьшая доля просроченных кредитов у заемщиков с высоким доходом (свыше 196 тыс.руб);
- 8,8% - наибольшая доля просроченных кредитов у заемщиков с доходом немного ниже среднего (от 108 тыс.руб. до 142,6 тыс.руб.)
        
Рекомендуется провести исследование на большей выборке данных.
        
**Фактор №4. Влияние фактора подтверждено.**
- 7,3% - доля просроченных кредитов с целью «операции с недвижимостью»;
- 7,9% - доля просроченных кредитов с целью «проведение свадьбы»;
- 9,3% - доля просроченных кредитов с целью «операции с автомобилем»;
- 9,3% - доля просроченных кредитов с целью «получение образования»;
        
Разница в доле просрочки между первым и последним пунктами 2 п.п. Таким образом, отсутствие зарегистрированных отношений увеличивает количество просроченных кредитов на 27,4%. 
При этом, необходимо учитывать, что данный фактор не является ключевым и единственным, поэтому рекомендуется устанавливать ему невысокий вес при принятии решения об уровне кредитоспособности.

**Общий вывод:**
1.	На текущий момент для построения модели кредитного скоринга **можно использовать факторы №2 и №4**
2.	Для более точной оценки влияния факторов №1 и №3 необходима более полная выборка.